In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'hvplot'

In [ ]:
# Load .env environment variables
env_path = Path("./alpaca_keys.env")
load_dotenv(dotenv_path=env_path)

In [ ]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [ ]:
# Create a connection to the API 
alpaca = tradeapi.REST(
    API_KEY,
    API_SECRET,
    ALPACA_API_BASE_URL,
    api_version="v2")

In [ ]:
# Set the tickers
eth_ticker = ["ETHUSD"]
btc_ticker = ["BTCUSD"]

In [ ]:
# Set the parameter tz to "America/New_York", 

start_date = pd.Timestamp("2016-06-08", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-02-08", tz="America/New_York").isoformat()

In [ ]:
# Set timeframe to one day (1D) for the Alpaca API
timeframe = "1Day"

In [ ]:
# Use the Alpaca get_crypto_bars function to gather the price information for ETHEREUM

eth_data_df = alpaca.get_crypto_bars(
    eth_ticker,
    timeframe,
    start = start_date,
    end = end_date,
    exchanges = "CBSE"
).df


# Review the resulting `portfolio_prices_df` DataFrame. 
display(eth_data_df.head())

In [ ]:
# Calculate the daily returns using the closing prices and the pct_change function
eth_data_df["actual returns"] = eth_data_df["close"].pct_change()

# Drop all NaN values from the DataFrame
eth_data_df = eth_data_df.dropna()

In [ ]:
eth_prices_df = eth_data_df[['symbol', 'open', 'high', 'low', 'close', 'volume', 'actual returns']]
display(eth_prices_df.head(10))
display(eth_prices_df.tail(10))


In [ ]:
eth_prices_df.index = eth_prices_df.index.date
eth_prices_df

In [ ]:
# Use the Alpaca get_crypto_bars function to gather the price information for BITCOIN
btc_data_df = alpaca.get_crypto_bars(
    btc_ticker,
    timeframe,
    start = start_date,
    end = end_date,
    exchanges = "CBSE"
).df

display(btc_data_df.head())

In [ ]:
# Calculate the daily returns using the closing prices and the pct_change function
btc_data_df["actual returns"] = btc_data_df["close"].pct_change()

# Drop all NaN values from the DataFrame
btc_data_df = btc_data_df.dropna()

In [ ]:
btc_prices_df = btc_data_df[['symbol', 'open', 'high', 'low', 'close', 'volume', "actual returns"]]

display(btc_prices_df.head(10))
display(btc_prices_df.tail(10))

In [ ]:
btc_prices_df.index = btc_prices_df.index.date
btc_prices_df

In [ ]:
## SMA_SVM Learning Method 

In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn import svm

In [ ]:
# Drop duplicate index values
eth_prices_df = eth_prices_df.loc[~eth_prices_df.index.duplicated(), :]
btc_prices_df = btc_prices_df.loc[~btc_prices_df.index.duplicated(), :]

In [ ]:
display(eth_prices_df.head(10))
display(eth_prices_df.tail(10))

In [ ]:
display(btc_prices_df.head(10))
display(btc_prices_df.tail(10))

In [ ]:
##Trading DataFrame

In [ ]:
# Creating trading DataFrames for both currencies
eth_trading_df = eth_prices_df.loc[:,['close', 'actual returns']]
btc_trading_df = btc_prices_df.loc[:,['close', 'actual returns']]

In [ ]:
# Rename Index to respective Currency
eth_trading_df.index.names = ['ETH']
btc_trading_df.index.names = ['BTC']

In [ ]:
# Set the variables for short window and long window periods
short_window = 200
long_window = 400


In [ ]:
# Generate the short and long window simple moving averages (20 and 200 days, respectively)
eth_trading_df["SMA200"] = eth_trading_df["close"].rolling(window=short_window).mean()
eth_trading_df["SMA400"] = eth_trading_df["close"].rolling(window=long_window).mean()

In [ ]:
# Generate the short and long window simple moving averages (10 and 20 days, respectively)
btc_trading_df["SMA200"] = btc_trading_df["close"].rolling(window=short_window).mean()
btc_trading_df["SMA400"] = btc_trading_df["close"].rolling(window=long_window).mean()

In [ ]:
# Drop null values in both DataFrames.
eth_trading_df = eth_trading_df.dropna()
btc_trading_df = btc_trading_df.dropna()

In [ ]:
# Create a new column in the `eth_trading_df` & 'btc_trading_df' called "signal" setting its value to zero.
eth_trading_df['signal'] = 0.0
btc_trading_df['signal'] = 0.0

In [ ]:
display(eth_trading_df.head(10))
display(btc_trading_df.head(10))



In [ ]:
# Assign a copy of the `SMA10` and `SMA50` columns to a new DataFrame called `eth_X`
eth_X = eth_trading_df[['SMA200', 'SMA400']].shift().dropna().copy()

# Display sample data
display(eth_X.head())
display(eth_X.tail())


In [ ]:
# Assign a copy of the `SMA10` and `SMA50` columns to a new DataFrame called `btc_X`
btc_X = btc_trading_df[['SMA200', 'SMA400']].shift().dropna().copy()

# Display sample data
display(btc_X.head())
display(btc_X.tail())

In [ ]:
# Create Target Set; Buy & Sell Signals 

In [ ]:
# Display sample data
display(btc_trading_df.head())
display(btc_trading_df.tail())

In [ ]:
# Create the signal to buy
eth_trading_df.loc[(eth_trading_df['SMA200'] > eth_trading_df['SMA400']), 'signal'] = 1

# Create the signal to sell
eth_trading_df.loc[(eth_trading_df['SMA200'] < eth_trading_df['SMA400']), 'signal'] = -1

In [ ]:
# Create the signal to buy
btc_trading_df.loc[(btc_trading_df['SMA200'] > btc_trading_df['SMA400']), 'signal'] = 1

# Create the signal to sell
btc_trading_df.loc[(btc_trading_df['SMA200'] < btc_trading_df['SMA400']), 'signal'] = -1

In [ ]:
# Display sample data
display(btc_prices_df.head(20))
display(btc_prices_df.tail())

In [ ]:
# Copy the new "signal" column to a new Series called `eth_y` & 'btc_y'.
eth_y = eth_trading_df['signal'].copy()
btc_y = btc_trading_df['signal'].copy()

eth_y.head()

In [ ]:
# Display sample data
display(btc_trading_df.head())
display(btc_trading_df.tail())

In [ ]:
eth_trading_df.describe()


In [ ]:
btc_trading_df.describe()

In [ ]:
##Create the Training and Testing Datasets

In [ ]:
from pandas.tseries.offsets import DateOffset

# Select the start of the training period
eth_training_dates = eth_prices_df.loc[pd.to_datetime("2016-06-11").date():training_enddate]
eth_testing_dates = eth_prices_df.loc[testing_startdate:testing_enddate]

# Display the training begin date
display(eth_training_dates.head())
display(eth_testing_dates.head())

In [ ]:
training_startdate = pd.to_datetime("2017-07-14").date()
training_enddate = pd.to_datetime( "2020-09-08").date()
testing_startdate = pd.to_datetime("2020-09-09").date()
testing_enddate = pd.to_datetime( "2022-02-08").date()

In [ ]:
# Generate the X_train and Y-train DataFrames
eth_X_train= eth_X.loc[training_startdate:training_enddate]
eth_y_train= eth_y.loc[training_startdate:training_enddate]

# Generate the Y_train DataFrame
btc_X_train= btc_X.loc[training_startdate:training_enddate]
btc_y_train= btc_y.loc[pd.to_datetime("2017-07-14").date():training_enddate]

# Display sample data
display(eth_X_train.head())

display(btc_X_train.head())



In [ ]:
eth_X_train.describe()

In [ ]:
eth_y_train.describe()

In [ ]:
## Creating Testing Datasets

In [ ]:
# Generate the X_test and y_test DataFrames for ETH & BTC
eth_X_test = eth_X.loc[testing_startdate:testing_enddate]
eth_y_test = eth_y.loc[testing_startdate:testing_enddate]

btc_X_test = btc_X.loc[testing_startdate:testing_enddate]
btc_y_test = btc_y.loc[testing_startdate:testing_enddate]


# Display sample data
eth_X_test.head()

btc_X_test.head()


In [ ]:
## Standardize the Data

In [ ]:
# Import the required module
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# Create a StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Apply the scaler model to fit the ETH & BTC X_train data
eth_X_scaler = scaler.fit(eth_X_train)
btc_X_scaler = scaler.fit(btc_X_train)

# Transform the ETH & BTC  X_train and X_test DataFrames using the X_scaler
eth_X_train_scaled = eth_X_scaler.transform(eth_X_train)
eth_X_test_scaled = eth_X_scaler.transform(eth_X_test)

btc_X_train_scaled = btc_X_scaler.transform(btc_X_train)
btc_X_test_scaled = btc_X_scaler.transform(btc_X_test)

print(eth_y_train.describe())


In [ ]:
# Create the classifier model
eth_svm_model = svm.SVC()
btc_svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
eth_svm_model = eth_svm_model.fit(eth_X_train_scaled, eth_y_train)
btc_svm_model = btc_svm_model.fit(btc_X_train_scaled, btc_y_train)

# Use the trained model to predict the trading signals for the training data
eth_training_signal_predictions = eth_svm_model.predict(eth_X_train_scaled)
btc_training_signal_predictions = btc_svm_model.predict(btc_X_test_scaled)

# Display the sample predictions
display(eth_training_signal_predictions[:10])
display(btc_training_signal_predictions[:10])

In [ ]:
np.min(eth_training_signal_predictions)


In [ ]:
## Classification Report 

In [ ]:
eth_training_report = classification_report(eth_y_train, eth_training_signal_predictions)
print(eth_training_report)

In [ ]:
## Backtesting

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
eth_testing_signal_predictions = eth_svm_model.predict(eth_X_test_scaled)
btc_testing_signal_predictions = btc_svm_model.predict(btc_X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
eth_testing_report = classification_report(eth_y_test, eth_testing_signal_predictions)
print(eth_testing_report)

btc_testing_report = classification_report(btc_y_test, btc_testing_signal_predictions)
print(btc_testing_report)

In [ ]:
eth_y_test.describe()

In [ ]:
## Compare Actual and Predicted Returns


In [ ]:
# Create a ETH predictions DataFrame
eth_predictions_df = pd.DataFrame(index=eth_X_test.index)

eth_predictions_df["predicted_signal"] = eth_testing_signal_predictions

eth_predictions_df["actual returns"] = eth_trading_df["actual returns"]

eth_predictions_df["trading_algorithm_returns"] = (
    eth_predictions_df["actual returns"] * eth_predictions_df["predicted_signal"]
)

# Review the DataFrame
eth_predictions_df.head()

In [3]:
# Create a BTC predictions DataFrame
btc_predictions_df = pd.DataFrame(index=btc_X_test.index)

btc_predictions_df["predicted_signal"] = btc_testing_signal_predictions

btc_predictions_df["actual returns"] = btc_trading_df["actual returns"]

btc_predictions_df["trading_algorithm_returns"] = (
    btc_predictions_df["actual returns"] * btc_predictions_df["predicted_signal"]
)

# Review the DataFrame
btc_predictions_df.head()

NameError: name 'btc_X_test' is not defined

In [ ]:
 ## Plot Cumulative Returns

In [ ]:
# Import 
import hvplot.pandas

In [65]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + eth_predictions_df[["actual returns", "trading_algorithm_returns"]]).cumprod().hvplot(title='ETH', width=1000, height=500)

:NdOverlay   [Variable]
   :Curve   [ETH]   (value)

In [2]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + btc_predictions_df[["actual returns", "trading_algorithm_returns"]]).cumprod().hvplot(title='BTC', width=1000, height=500)

NameError: name 'btc_predictions_df' is not defined